In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### Dados

In [ ]:
# Importar dados
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSr7tU5tK8cvvR32yypE1PArgXhmNbFJ9bw8w6Sm2zQhyaMs27csoo-77vgFedHw9z25Ez3Qm-geKyU/pub?gid=43084471&single=true&output=csv', skiprows= 1)

df.drop(columns='Unnamed: 0', inplace= True)
df.reset_index(drop = True)
df

### Função para fazer os plots

In [ ]:
def plotar(coluna1, coluna2):

    df[coluna1] = df[coluna1].astype(str).str.replace(',', '').str.strip()
    df[coluna2] = df[coluna2].astype(str).str.replace(',', '').str.strip()

    # Converter para float
    df[coluna1] = pd.to_numeric(df[coluna1], errors='coerce')
    df[coluna2] = pd.to_numeric(df[coluna2], errors='coerce')

    X = df[coluna1].values
    Y = df[coluna2].values
    
    # Ajuste polinomial de grau 1
    Z = np.polyfit(X, Y, 1)
    p = np.poly1d(Z)
    print(f"Relação entre os parâmetros: {coluna2} = {p[0].round(3)} + {p[1].round(3)} {coluna1}")
    
    # Predição de valores
    Y_pred = p(X)
    delta_x = (X.max() - X.min()) / 60
    delta_y = (Y.max() - Y.min()) / 25
    
    # DataFrame auxiliar
    df_aviao = df[['Parameter', coluna1, coluna2]].copy()
    df_aviao['Y_pred'] = Y_pred
    df_aviao.set_index('Parameter', inplace= True)
    
    plt.figure(figsize=(12, 8))
    
    # Plot dos dados
    plt.scatter(X, Y, label='Dados Históricos', color = 'black')

    # Plot da Regressão
    sns.set_theme(style="white")
    sns.lineplot(x=np.sort(X), y=np.sort(Y_pred), color='gray', linestyle='dashed', label=f'Regressão Linear: f(x) = {p[0].round(3)} + {p[1].round(6)}x')
    
    # Legenda nos pontos
    for index, aviao in df_aviao.iterrows():
        x = aviao[coluna1]
        y = aviao[coluna2]
        y_pred = aviao['Y_pred']
        label_x = x - delta_x
        label_y = y + delta_y*0.5 if y > y_pred else y - delta_y
        plt.text(label_x, label_y, str(index), fontsize=10)

    # Legenda e nome dos eixos
    plt.xlabel(coluna1, fontsize=16)
    plt.ylabel(coluna2, fontsize=16)
    plt.legend(fontsize = 14)
    
    ax = plt.gca()

    # Remover os eixos superior e direito
    ax.tick_params(axis='x', top=False)
    ax.tick_params(axis='y', right=False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Setar espessura dos eixos (inferior e esquerdo) e adicionar os traços
    ax.spines['bottom'].set_linewidth(0.6)
    ax.spines['left'].set_linewidth(0.6)
    ax.tick_params(axis='x', bottom=True, direction='in', length=4, width=0.6)
    ax.tick_params(axis='y', left=True, direction='in', length=4, width=0.6)
    
    plt.grid(False)
    
    plt.show()


In [ ]:
# Peso Vazio x MTOW

plotar("MTOW", "Empty_Weight")


Relação entre os parâmetros: Empty_Weight = 996.768 + 0.559 MTOW


In [ ]:
# Número de passageiros x MTOW

In [ ]:
# Número de passageiros x Comprimento de fuselagem